In [1]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst

In [2]:
import random
import struct
import numpy
import os, os.path, shutil

**GDAL/OGR** - либы для чтения/записи геоданных

Библиотека единой абстрактной модели геоданных GDAL, первоначально была просто библиотекой для работы с растровыми геоданными, в то время как библиотека 0GR предназначалась для работы с векторными данными.

Теперь либы объединены. Стандартная версия gdal позволяет читать в 100 форматах, а писать в 71. ogr читает вы 42 форматах, пишет в 39.

Для чтения и записи данных в растровом формате библиотека GDAL исполь?
зует специальный объект dataset.

![](img/001.png)

- размер растра - общая ширина в пикселах и высота в строках. Каждая ячейка привязана географически - ей присвоены координаты в результате применения некой модели трансформации
- система координат включает картографическую проекцию, геодезический датум, единицы измерения и масштаб
- метадата содержит доп.информацию о наборе
- размер растрового канала - размер пикселов в ширину и линий в высоту для данных внутри канала.
- таблица цветности описывает как значения пикселов транислируются в цвет

каждая ячейка содержит целое число и число с плавающей точкой. gdal поддерживает отсутствие значяений

Так как у каждого растрового канала обычно имеются тысячи (и даже миллионы) ячеек, то к ним обычно обращаются не поэлементно. Вместо этого для получения доступа ко всем данным канала целиком можно воспользоваться одним из двух возможных методов:

- можно воспользоваться методом band. ReadRaster (), чтобы прочитать часть или все данные канала в двоичную строку и затем для конвертирования этой строки в массив значений использовать встроенную библиотеку struct;
- можно воспользоваться методом band.ReadArray(), чтобы прочитать часть данных канала, сохраняя значения ячеек непосредственно в объект-массив библиотеки NumPy.

#### Запись  растровых данных

Рассмотрим использование библиотеки GDAL для записи растровых данных в файл формата GeoTIFF. Для примера разделим всю поверхность Земли на 360 ячеек по горизонтали и 180 ячеек по вертикали, с тем чтобы каждая ячейка покрывала один градус широты и долготы. Для каждой ячейки сгенерируем одно случайное число между 1 и 100.

In [3]:
driver = gdal.GetDriverByName("GTiff")
dstFile = driver.Create("output/example.tiff", 
                        360, 180, 1, gdal.GDT_Int16)

Метод gdal.driver.Create() принимает следующие параметры:

- имя растрового файла; 
- нужное число ячеек по горизонтали; 
- нужное число ячеек по вертикали; 
- нужное число каналов в файле; 
- константа, задающая тип информации, хранимой в каждой ячейке. В на шем случае каждая ячейка будет содержать 16-разрядное целочисленное значение.

In [4]:
# создадим проекцию что бы спозиционировать 
# ячейки на поверхность Земли
# Вспользуемся датумом WGS84
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS("WGS84")
dstFile.SetProjection(spatialReference.ExportToWkt())

0

модель трансформации данных задается списком из шести числе (матрица афинных преобразований)

Мы можем проигнорировать математику аффинных трансформаций и задать матрицу при помощи всего четырех значений: 

- позиция Х и Y верхнего левого угла верхней левой ячейки; 
- ширина каждой ячейки, измеряемая в градусах долготы; 
- высота каждой ячейки, измеряемая в градусах широты

In [5]:
# зададим модель трансформации
originX = -180
originY = 90
cellWidth = 1.0
cellHeight = 1.0

dstFile.SetGeoTransform([originX, cellWidth, 0,
                         originY, 0, -cellHeight])

0

In [6]:
# ссылка на объект gdal.Band для сохраненеия данных в растровом канале
band = dstFile.GetRasterBand(1)

In [7]:
# сгенерируем данные для растрового канала
values = []
for row in range(180):
    row_data = []
    for col in range(360):
        row_data.append(random.randint(1, 100))
    values.append(row_data)

In [8]:
# пишем построчно в файл через стракт
# fmt = "<" + ("h" * band.XSize)

# for row in range(180):
#     scanline = struct.pack(fmt, *values[row])
#     band.WriteRaster(0, row, 360, 1, scanline)

In [9]:
# другой вариант - пишем целиком массив нампи
array = numpy.array(values, dtype=numpy.int16)
band.WriteArray(array)

0

In [10]:
# нужно закерывать файл (или присваивать ему None)
del dstFile 

#### Чтение растровых данных

In [11]:
srcFile = gdal.Open("output/example.tiff")
band = srcFile.GetRasterBand(1)

In [12]:
# чтение для стракт
# fmt = "<" + ("h" * band.XSize)

# for row in range(band.YSize):
#     scanline = band.ReadRaster(0, row, band.XSize, 1,
#                                band.XSize, 1,
#                                band.DataType)
#     row_data = struct.unpack(fmt, scanline)
#     print(row_data)

In [13]:
# чтение для numpy
values = band.ReadAsArray()
for row in range(band.XSize):
    print(values[row])

[ 41  40  89  43  16  94  80  81  29  55  38  49  11  34   9   6  37  17
  26  72  73  11  56  64  87  94  13  72   9  83  72  82  72  73  56  17
  89  83  70  81  67  83  72  95  96  25  80   4   6  42  64  82  23   1
  98  67   3  22  53  25  41  61  18   6  50  61  77  25  18  99  36  10
  24  23  87  55   8  99  23   5  19   2  81  67  17  20   1  21  56  80
  80   4  10  23  74   9  71  13  28  50  30  18  35  96  94  81  73  34
  59  75  90   6  45  77  22  97  39  68  26   6   9  60  97  10  11   8
   6  61  65  59 100  18  60  80  48   5  33   6  67  90  82  69   7  37
  30  88  85  43  35  84  99   1  98  89  20  11  91  79   4  80  77  65
  12  10  33  49  65  78  40  78  15  94  84  59  91  91   3  43  28  84
  22  24  85  88  20  58  82  96  67  35  84  71  82  12  65  20  76  78
  69  94  77  68  79  19  60  63   6  55  58  10  43 100   3  67  81  81
   6  40   7   2  66  60  15  66  47  90   2  63  34  84  18 100  20  15
  53  44  99  95  37  19  43  99   7  23  33  21  9

IndexError: index 180 is out of bounds for axis 0 with size 180

Структура данных в растре

![](img/002.png)

#### Запись векторных данных

Сгенерируем набор произвольных геометрий точки и сохраним их в файле фигур вместе с некоторой атрибутивной информацией.

In [ ]:
if os.path.exists("output/test-shapefile"):
    shutil.rmtree("output/test-shapefile")
os.mkdir("output/test-shapefile")

In [ ]:
# зададим драйвер
driver = ogr.GetDriverByName("ESRI Shapefile")
path = os.path.join("output/test-shapefile", "shapefile.shp")
datasource = driver.CreateDataSource(path)

In [ ]:
# определим слой
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS('WGS84')

In [ ]:
# создадим слой
layer = datasource.CreateLayer("layer", spatialReference)

In [ ]:
# добавим в слой типы данных и атрибуты, 
# которые мы планируем там хранить
field = ogr.FieldDefn("ID", ogr.OFTInteger)
field.SetWidth(4)
layer.CreateField(field)

field = ogr.FieldDefn("NAME", ogr.OFTString)
field.SetWidth(20)
layer.CreateField(field)

In [ ]:
# добавим немного данных в слой
for i in range(100):
    id_ = 1000 + i
    lat  = random.uniform(-90, +90)
    long = random.uniform(-180, +180)
    name = "point-{}".format(i)

    # тут определим геометрию точки
    wkt = "POINT({} {})".format(long, lat)    
    geometry = ogr.CreateGeometryFromWkt(wkt)

    # для каждой точки создадим объект ogr.Feature
    # и сохраним в нем все данные
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetGeometry(geometry)
    feature.SetField("ID", id_)
    feature.SetField("NAME", name)

    # разместим объект в слове
    layer.CreateFeature(feature)

In [ ]:
# осовобождаем ресурсы
feature.Destroy()
datasource.Destroy()
del datasource

#### Чтение векторых ф-лов

In [ ]:
shapefile = ogr.Open("output/test-shapefile/shapefile.shp")
layer = shapefile.GetLayer(0)

for i in range(layer.GetFeatureCount()):
    feature = layer.GetFeature(i)
    id_ = feature.GetField("ID")
    name = feature.GetField("NAME")
    geometry = feature.GetGeometryRef()
    print(i, id_, name, geometry.GetX(), geometry.GetY())

#### Работа с проекциями в pyproj

![](img/003.png)

Класс Proj выполняет преобразования значений долготы и широты в прямо?
угольные координаты (х, у) и наоборот, тогда как класс Geod выполняет различные расчеты на дуге большого круга и углов. Оба реализованы поверх динамической библиотеки PR0J.4

In [ ]:
import pyproj
print(pyproj.__version__)

Пример использования pyproj

Указание географического положения в координатах 17-й зоны проекции Меркатора (UTM), затем при помощи двух объектов Proj, один с заданной 17-й зоной UTM и другой с координатной проекцией (широта и долгота), выполняет перевод прямоугольных координат этого географического положения в значения широты и долготы

In [ ]:
UTM_X = 565718.5235
UTM_Y = 3980998.9244

srcProj = pyproj.Proj(proj="utm", zone="11", ellps="clrk66", units="m")
dstProj = pyproj.Proj(proj="longlat", ellps="WGS84", datum="WGS84")

long,lat = pyproj.transform(srcProj, dstProj, UTM_X, UTM_Y)

print("UTM zone 17 coordinate " +
      "({:.4f}, {:.4f}) ".format(UTM_X, UTM_Y) +
       "= {:.4f}, {:.4f}".format(long, lat))

**В pyproj2 новый пайплайн!!!**

[изменеения в версиях pyproj](https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1)

In [ ]:
from pyproj import Transformer

transformer = Transformer.from_crs("epsg:4326", "epsg:3857")
transformer.transform(12, 12)

Второй пример принимает эти расчетные значения широты и долготы и при помощи объекта Geod вычисляет еще одну точку в 10 км к северо-востоку от этого географического положения:

In [ ]:
angle   = 315 # 315 degrees = northeast.
distance = 10000

geod = pyproj.Geod(ellps="WGS84")
long2,lat2,invAngle = geod.fwd(long, lat, angle, distance)

print("{:.4f}, {:.4f}".format(lat2, long2) +
      " is 10km northeast of " +
      "{:.4f}, {:.4f}".format(lat, long))

#### Геоанализ и геообработка с помошью shapely

Библиотека Shapely разделена на ряд отдельных модулей, которые импортируются по мере необходимости. Наиболее часто используемые модули следующие: 
- shapely.geometry: задает все используемые в Shapely классы базовых геометрических фигур;
- shapely.wkt: предоставляет функции конвертирования объектов геометрии Shapely в строки в формате стандартного текстового представления WKT и наоборот;
- shapely.wkb: предоставляет функции конвертирования объектов геометрии Shapely в двоичные данные в формате стандартного двоичного представления WKB и наоборот;
- shapely. ops: предоставляет функции для выполнения пакетных операций на ряде объектов геометрии

Библиотека Shapely поддерживает 8 фундаментальных типов геометрий, или геометрических примитивов:

![](img/005.png)

- shapely.geometry.Point (точка) изображает отдельную точку в пространстве. Точки могут быть двумерными (х, у) или трехмерными (х, у, z);
- shapely.geometry.LineString (полилиния) изображает ломаную, то есть последовательность точек, объединенных в линию. Ломаные линии могут быть простыми (без пересекающихся отрезков) или сложными (где два отрезка ломаной самопересекаются);
- shapely.geometry.LinearRing (линейное кольцо) изображает ломаную линию, где первая и последняя координаты совпадают. Отрезки внутри линейного кольца не могут самопересекаться или касаться друг друга;
- shapely.geometry.Polygon (многоугольник) изображает заполненную область, внутри которой дополнительно может быть одна или несколько «дыр»;
- shapely.geometry.Multipoint (мультиточка) изображает составную точку, то есть коллекцию точек;
- shapely.geometry.MultiLineString (мультиполилиния) изображает составную ломаную, то есть коллекцию ломаных линий;
- shapely.geometry.MultiPolygon (мультимногоугольник) изображает составной многоугольник, то есть коллекцию многоугольников;
- shapely.geometry.GeometryCollection (коллекция геометрий) представляет коллекцию, состоящую из любой комбинации точек, линий, линейных колец и многоугольников.

Shapely - это библиотека управления пространственными, а не
геопространственными данными. В ней не предусмотрено понятие географических координат. Вместо этого в ней предполагается, что геоданные, прежде чем ими начнут управлять, уже спроецированы, то есть перенесены, на двумерную координатную плоскость, и результаты при желании могут быть конвертированы в географические координаты

In [ ]:
# создает два геометрических объекта Shapely - круг и квадрат - 
# и вычисляет их пересечение. 
# Пересечением является многоугольник в форме четверти круга

import shapely.geometry
import shapely.wkt

pt = shapely.geometry.Point(0, 0)
circle = pt.buffer(1.0)

square = shapely.geometry.Polygon([(0, 0), (1, 0),
                                   (1, 1), (0, 1),
                                   (0, 0)])

intersect = circle.intersection(square)

for x,y in intersect.exterior.coords:
    print(x,y)
print(shapely.wkt.dumps(intersect))

#### Визуализация геоданных mapnik

In [18]:
import mapnik
print(mapnik.__file__)

None


In [19]:
help(mapnik)

Help on package mapnik:

NAME
    mapnik

PACKAGE CONTENTS
    config

FILE
    (built-in)




In [20]:
m = mapnik.Map(1024, 768)

AttributeError: module 'mapnik' has no attribute 'Map'